In [1]:
%pip install matplotlib seaborn plotly
%pip install ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 67.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from mechir.plotting import plot_components
import os
import numpy as np

ModuleNotFoundError: No module named 'mechir'

In [ ]:
DIR = r'../data/data/patching/'

In [ ]:
out = {}
for file in os.listdir(DIR):
    out[file] = np.load(DIR + file)

In [ ]:
for file in out:
    plot_components(out[file], title=file)